In [1]:
# In this notebook we evaluate the Wordnet based ingredients extraction 

from glob import glob
import pandas as pd

# first import all GS files and create a nice datastructure
df = pd.DataFrame(columns=['id','category','values'])
for name in glob('/data/IngredientQuantitiesUnits_Annotations/EvaluationSet/ba*/*csv'):
    name_elems = name.split("/")
    file2 = pd.read_csv(name)
    for hip, hop in file2.iterrows():
        if hop['TYPE'] == "PERSON":
            row2 = [name_elems[9][:-4], 'INGREDIENTS', name_elems[9][:-4]+"_"+hop["QUOTE_TRANSCRIPTION"].lower() ] 
            df.loc[len(df) ] = row2
            row2 = []
        if hop['TYPE'] == "PLACE":
            row3 = [name_elems[9][:-4],  'QUANTITIES', name_elems[9][:-4]+"_"+hop["QUOTE_TRANSCRIPTION"].lower() ] 
            df.loc[len(df) ] = row3
            row3 = []
        if hop['TYPE'] == "EVENT":
            row4 = [name_elems[9][:-4],  'OCR', name_elems[9][:-4]+"_"+hop["QUOTE_TRANSCRIPTION"].lower() ] 
            df.loc[len(df)  ] = row4
            row4 = []

In [2]:
huppeldepupje = pd.DataFrame(columns=['id','category', 'ingr_set'])
huppeldepupje = df.groupby(['id','category'])['values'].apply(set)

In [3]:
newframe = huppeldepupje.to_frame()
newnewframe = newframe.reset_index(level='category')
newframe = newnewframe.reset_index(level='id')

In [4]:
from pattern.nl import pluralize, singularize

def import_list():
## IMPORT NEW LIST
# import the list of ingredients
# Complete list seems to result in a slightly higher recall
    list_ = []
    #with open("ingredient_list.csv", 'r') as x: # yes it's hard coded, not so neat
    with open("/data/ingredients_lists/wordnet_ingredients.txt", 'r') as x: # yes it's hard coded, not so neat
        for line in x:
            line = line.rstrip()
            list_.append(pluralize(line))
            list_.append(line)
#	print(list_)
    return(list_)

def preprocess_text(t):
    tagged_list_ = []
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(t)
    words = " ".join(words)
    return(words)

def detect_ingredients(text,lijst):
    ingredients = []
    for x in lijst:
        x = " " + x +" "
        ingredient = re.search(x, text)
        if ingredient:
            cleanright = ingredient.group().rstrip()
            cleanleft = cleanright.lstrip()
            lowercase = cleanleft.lower()            
            ingredients.append(lowercase)
        unique_ingredients = set(ingredients)
#	print("bla", unique_ingredients)
    return(unique_ingredients)

wnlist = import_list()

whl = open("wnlist_expanded.txt" ,'w')
for x in wnlist:
    whl.write(x + "\n")

whl.close()

In [5]:
import re
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from glob import glob

system_output = {} 
for filename in glob('/data/IngredientQuantitiesUnits_Annotations/EvaluationSet/ba*/*txt'):
    file = open(filename, "r") 
    name_elems = filename.split("/")
    artikel = file.read()

    ing = preprocess_text(artikel)
    #print(ing)
    found = detect_ingredients(ing, wnlist)
#	qua = detect_quantities(ing, found)
    found_list = list(found)
    inbetween = []
    for x in found_list:
        inbetween.append(name_elems[9][:-4] + "_" + x)
    found = set(inbetween)
    inbetween = []
    system_output[name_elems[9][:-4]] = found
    found = {}

In [6]:
aggregated = pd.DataFrame(columns=['category', 'ingr_set'])
aggregated = df.groupby(['category'])['values'].apply(set)
aggframe = aggregated.to_frame()
aggframe_reset = aggframe.reset_index(level='category')


In [7]:
sysout_aggregated = []
for x in system_output:
    for y in system_output[x]:
        sysout_aggregated.append(y)
        
sysout_agg = set(sysout_aggregated)

In [8]:
def prec_r_f(a1,a2):
    tp = a1.intersection(a2)
    fp = a2.difference(a1)
    fn = a1.difference(a2)
    precision = len(tp) / (len(tp) + len(fp))
    recall = len(tp) / (len(tp) + len(fn)) 
    f = (precision + recall) / 2 
    answer = "\tItemsGS:\t" + str(len(a1)) + "\tItemsSystem:\t" + str(len(a2)) + "\tcorrect:\t" + str(len(tp))+ "\tp:\t" + str(precision) + "\tr:\t" + str(recall) + "\tf1:\t" + str(f) + "\nin SystemOutput not in GS" + str(fp) + "\nin GS not in SystemOutput" + str(fn) 
    return(answer)

In [9]:
#gs_agg_ingr = aggframe_reset.loc[(aggframe_reset['category'] == 'INGREDIENTS') ]['values'].tolist()
#agg_list_ingr = set(gs_agg_ingr[0])
#prf_agg_ingr = prec_r_f(agg_list_ingr,sysout_agg)
#print(prf_agg_ingr)

gs_agg_ingr = aggframe_reset.loc[(aggframe_reset['category'] == 'INGREDIENTS')  ]['values'].tolist()
gs_agg_ocr = aggframe_reset.loc[(aggframe_reset['category'] == 'OCR')  ]['values'].tolist()
gs_agg_ocr_ing = (gs_agg_ocr[0] | gs_agg_ingr[0])
#agg_list_ingr = set(gs_agg_ingr[0])
prf_agg_ingr = prec_r_f(gs_agg_ocr_ing,sysout_agg)
print(prf_agg_ingr)

	ItemsGS:	1686	ItemsSystem:	1236	correct:	764	p:	0.6181229773462783	r:	0.4531435349940688	f1:	0.5356332561701735
in SystemOutput not in GS{'5366_gerecht', '4411_citroensap', '3840_soep', '3668_wit', '945_zakken', '1815_wijn', '945_witten', '2063_sap', '1343_levensmiddelen', '3395_koekjes', '324_glazuur', '945_dooiers', '995_peper', '4904_kruiden', '1389_soep', '3370_alcohol', '5366_mede', '6842_bouillon', '4997_marinade', '874_saus', '2226_wijn', '2706_schotel', '7034_kloppen', '6428_koffie', '3393_gelei', '2825_wijn', '5259_beslag', '1934_deeg', '6601_rassen', '6694_den', '6694_glazuur', '1934_koekje', '945_happen', '4602_puree', '4978_gerecht', '7419_wijn', '1363_smaakmaker', '1389_pap', '6401_graansoort', '5004_azijn', '7034_crème', '4622_perzik', '7126_saus', '6767_gehakt', '372_saus', '1389_salade', '3553_omdraaien', '2686_kaas', '6056_wit', '4391_kruidnagel', '5340_fris', '6401_wit', '3686_salade', '4082_bloem', '6401_rijst', '7419_saus', '5147_kruiden', '320_citroen', '1716_broo

In [12]:
print(len(wnlist))

3204


In [10]:
def import_list():
## IMPORT NEW LIST
# import the list of ingredients
# Complete list seems to result in a slightly higher recall
    list_ = []
    #with open("ingredient_list.csv", 'r') as x: # yes it's hard coded, not so neat
    with open("/data/IngredientQuantitiesUnits_Annotations/ingredients_lists/combined_AH_dbp_WN_list.txt", 'r') as x: # yes it's hard coded, not so neat
        for line in x:
            line = line.rstrip()
            list_.append(line)
#	print(list_)
    return(list_)

combined_list = import_list()

In [11]:
def preprocess_text(t):
    tagged_list_ = []
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(t)
    words = " ".join(words)
    return(words)

def detect_ingredients(text,lijst):
    ingredients = []
    for x in lijst:
        x = " " + x +" "
        ingredient = re.search(x, text)
        if ingredient:
            cleanright = ingredient.group().rstrip()
            cleanleft = cleanright.lstrip()
            lowercase = cleanleft.lower()            
            ingredients.append(lowercase)
        unique_ingredients = set(ingredients)
#	print("bla", unique_ingredients)
    return(unique_ingredients)

In [12]:
import re
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from glob import glob

system_output = {} 
for filename in glob('/data/IngredientQuantitiesUnits_Annotations/EvaluationSet/ba*/*txt'):
    file = open(filename, "r") 
    name_elems = filename.split("/")
    artikel = file.read()

    ing = preprocess_text(artikel)
    #print(ing)
    found = detect_ingredients(ing, combined_list)
#	qua = detect_quantities(ing, found)
    found_list = list(found)
    inbetween = []
    for x in found_list:
        inbetween.append(name_elems[9][:-4] + "_" + x)
    found = set(inbetween)
    inbetween = []
    system_output[name_elems[9][:-4]] = found
    found = {}

In [13]:
aggregated = pd.DataFrame(columns=['category', 'ingr_set'])
aggregated = df.groupby(['category'])['values'].apply(set)
aggframe = aggregated.to_frame()
aggframe_reset = aggframe.reset_index(level='category')

In [14]:
sysout_aggregated = []
for x in system_output:
    for y in system_output[x]:
        sysout_aggregated.append(y)
        
sysout_agg = set(sysout_aggregated)

In [15]:
def prec_r_f(a1,a2):
    tp = a1.intersection(a2)
    fp = a2.difference(a1)
    fn = a1.difference(a2)
    precision = len(tp) / (len(tp) + len(fp))
    recall = len(tp) / (len(tp) + len(fn)) 
    f = (precision + recall) / 2 
    answer = "\tItemsGS:\t" + str(len(a1)) + "\tItemsSystem:\t" + str(len(a2)) + "\tcorrect:\t" + str(len(tp))+ "\tp:\t" + str(precision) + "\tr:\t" + str(recall) + "\tf1:\t" + str(f) + "\nin SystemOutput not in GS" + str(fp) + "\nin GS not in SystemOutput" + str(fn) 
    return(answer)

In [16]:
#gs_agg_ingr = aggframe_reset.loc[(aggframe_reset['category'] == 'INGREDIENTS') ]['values'].tolist()
#agg_list_ingr = set(gs_agg_ingr[0])
#prf_agg_ingr = prec_r_f(agg_list_ingr,sysout_agg)
#print(prf_agg_ingr)

gs_agg_ingr = aggframe_reset.loc[(aggframe_reset['category'] == 'INGREDIENTS')  ]['values'].tolist()
gs_agg_ocr = aggframe_reset.loc[(aggframe_reset['category'] == 'OCR')  ]['values'].tolist()
gs_agg_ocr_ing = (gs_agg_ocr[0] | gs_agg_ingr[0])
#agg_list_ingr = set(gs_agg_ingr[0])
prf_agg_ingr = prec_r_f(gs_agg_ocr_ing,sysout_agg)
print(prf_agg_ingr)

	ItemsGS:	1686	ItemsSystem:	2050	correct:	1154	p:	0.5629268292682926	r:	0.6844602609727165	f1:	0.6236935451205046
in SystemOutput not in GS{'3395_zout en peper', '1815_uiensoep', '5366_gerecht', '4411_citroensap', '3840_soep', '3668_wit', '4710_oliebollen', '1815_wijn', '3553_groenten', '6767_thaise vissaus', '2063_sap', '1343_levensmiddelen', '1911_smaak', '3395_koekjes', '324_glazuur', '3678_chinese bieslook', '4612_water', '995_peper', '6842_groenten', '5341_bonensoep', '4904_kruiden', '1389_soep', '7417_rijstwijn', '3370_alcohol', '1901_kokend water', '5366_mede', '6056_zout en peper', '6842_bouillon', '4997_marinade', '7266_water', '1716_witte', '874_saus', '5147_gemalen kurkuma', '2226_wijn', '3840_zout en peper', '372_vet', '798_zon', '3475_zon', '2706_schotel', '7034_kloppen', '2825_franse', '7126_taart', '6541_witte bonen', '6428_koffie', '945_zoet', '3393_kruidenazijn', '1563_hollandse', '6401_zwarte peper', '1150_kabeljauw', '3393_gelei', '3370_water', '2825_wijn', '5259_bes